In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model, cluster, ensemble, metrics, model_selection
import xgboost
import catboost
import lightgbm

In [2]:
train = pd.read_pickle('dataset/train.pk')
val = pd.read_pickle('dataset/val.pk')
test = pd.read_pickle('dataset/test.pk')

In [3]:
train.shape, test.shape

((29370, 90), (13184, 89))

In [4]:
train.select_dtypes(exclude='number')

""
id
31077
307
37453
17537
22765
...
14481
35668
25986


In [5]:
val.select_dtypes(exclude='number')

""
id
20961
34281
19415
20810
31969
...
4653
32482
894


In [6]:
test.select_dtypes(exclude='number')

""
id
39162
39163
39164
39165
39166
...
52341
52342
52343


In [7]:
train.isna().sum()

campaign_var_1             0
campaign_var_2             0
products_purchased         0
user_activity_var_1        0
user_activity_var_2        0
                          ..
campaign_var_1_trim_tme    0
campaign_var_2_trim_tme    0
user_act_trim_tme          0
products_purchased_tme     0
created_at_month_tme       0
Length: 90, dtype: int64

In [8]:
target = 'buy'

In [9]:
y_train = train.pop(target)
X_train = train.copy()

In [10]:
y_val = val.pop(target)
X_val = val.copy()

In [11]:
# y_test = test.pop(target)
X_test = test.copy()

In [12]:
# X_train, X_val, y_train, y_val = model_selection.train_test_split(X, y, stratify=y, random_state=42)
# X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [13]:
total = y_train.shape[0]
pos = y_train.sum()
neg = total-pos

class_weights = [neg/total, pos/total]
class_weights

[0.9489955737146748, 0.051004426285325165]

In [14]:
pos_weight = neg/pos
pos_weight

18.60614152202937

In [15]:
model = linear_model.LogisticRegression(n_jobs=-1, verbose=2)
model = ensemble.RandomForestClassifier(
    n_estimators=1000, class_weight='balanced',
    max_depth=16, max_features=0.6, max_samples=0.8,
    n_jobs=-1, verbose=2, random_state=42
)
model = xgboost.XGBClassifier(
    n_estimators=1000, n_jobs=-1,
    scale_pos_weight=pos_weight, max_depth=5, verbosity=2
)
# model = lightgbm.LGBMClassifier(n_jobs=-1, verbose=2)
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, importance_type=None, interaction_constraints=None,
              learning_rate=None, max_cat_to_onehot=None, max_delta_step=None,
              max_depth=5, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None,
              scale_pos_weight=18.60614152202937, subsample=None,
              tree_method=None, ...)

In [16]:
model.fit(X_train, y_train)

[02:12:06] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 34 extra nodes, 0 pruned nodes, max_depth=5
[02:12:06] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=5
[02:12:06] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 0 pruned nodes, max_depth=5
[02:12:06] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 34 extra nodes, 0 pruned nodes, max_depth=5
[02:12:06] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 36 extra nodes, 0 pruned nodes, max_depth=5
[02:12:06] INFO: /home/roshande/install/

[02:12:08] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 58 extra nodes, 0 pruned nodes, max_depth=5
[02:12:08] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 0 pruned nodes, max_depth=5
[02:12:08] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=5
[02:12:08] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[02:12:09] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 0 pruned nodes, max_depth=5
[02:12:09] INFO: /home/roshande/install/

[02:12:11] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=5
[02:12:11] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 16 extra nodes, 0 pruned nodes, max_depth=5
[02:12:11] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 36 extra nodes, 0 pruned nodes, max_depth=5
[02:12:11] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=5
[02:12:11] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 0 pruned nodes, max_depth=5
[02:12:11] INFO: /home/roshande/install/

[02:12:13] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=5
[02:12:13] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 0 pruned nodes, max_depth=5
[02:12:13] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 36 extra nodes, 0 pruned nodes, max_depth=5
[02:12:13] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=5
[02:12:13] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 0 pruned nodes, max_depth=5
[02:12:13] INFO: /home/roshande/install/

[02:12:15] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=5
[02:12:15] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 36 extra nodes, 0 pruned nodes, max_depth=5
[02:12:15] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[02:12:15] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 52 extra nodes, 0 pruned nodes, max_depth=5
[02:12:15] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=5
[02:12:15] INFO: /home/roshande/install/

...

[02:12:35] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=5
[02:12:35] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=5
[02:12:35] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 0 pruned nodes, max_depth=5
[02:12:36] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 0 pruned nodes, max_depth=5
[02:12:36] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 26 extra nodes, 0 pruned nodes, max_depth=5
[02:12:36] INFO: /home/roshande/install/

[02:12:37] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=5
[02:12:37] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=5
[02:12:37] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=5
[02:12:37] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[02:12:37] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=5
[02:12:37] INFO: /home/roshande/install/

[02:12:38] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=5
[02:12:38] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[02:12:39] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=5
[02:12:39] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=5
[02:12:39] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=5
[02:12:39] INFO: /home/roshande/install/

[02:12:40] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 0 pruned nodes, max_depth=5
[02:12:40] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 36 extra nodes, 0 pruned nodes, max_depth=5
[02:12:40] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=5
[02:12:40] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=5
[02:12:40] INFO: /home/roshande/install/xgboost/python-package/build/temp.linux-x86_64-3.7/xgboost/src/tree/updater_prune.cc:101: tree pruning end, 36 extra nodes, 0 pruned nodes, max_depth=5
[02:12:40] INFO: /home/roshande/install/

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=-1, num_parallel_tree=1,
              predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=18.60614152202937, subsample=1,
              tree_method='exact', ...)

In [17]:
feat_imp = pd.Series(model.feature_importances_, index=X_train.columns)
feat_imp.sort_values(ascending=False)

signup_date_presence       0.253242
user_activity_var_9        0.163832
user_activity_var_2        0.076857
user_activity_var_8        0.062994
user_act_trim_tme          0.056993
                             ...   
user_act_trim_010010000    0.001193
user_act_trim_000011100    0.000000
user_act_trim_100001100    0.000000
user_act_trim_000010100    0.000000
user_act_trim_000001100    0.000000
Length: 89, dtype: float32

In [18]:
feat_imp.nlargest(15)*100

signup_date_presence       25.324211
user_activity_var_9        16.383230
user_activity_var_2         7.685687
user_activity_var_8         6.299366
user_act_trim_tme           5.699313
user_activity_var_11        1.626388
user_activity_var_4         1.383336
user_act_trim_100001000     1.116035
campaign_var_1_trim_11      0.965846
user_activity_var_10        0.943755
user_act_trim_000011010     0.878466
campaign_var_2              0.878335
user_act_trim_110010000     0.859635
products_purchased_tme      0.826908
user_act_trim_100010010     0.732472
dtype: float32

# Train scoring

In [19]:
y_train_prob = model.predict_proba(X_train)
y_train_prob

array([[9.9997711e-01, 2.2858734e-05],
       [9.9994886e-01, 5.1130428e-05],
       [9.9999982e-01, 1.7415380e-07],
       ...,
       [9.9702674e-01, 2.9732594e-03],
       [9.9975663e-01, 2.4337064e-04],
       [9.9999946e-01, 5.2162505e-07]], dtype=float32)

In [20]:
y_train_pred = model.predict(X_train)
y_train_pred

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
metrics.f1_score(y_train, y_train_pred)

0.9993328885923949

# Validation scoring

In [22]:
y_val_prob = model.predict_proba(X_val)
y_val_prob

array([[9.9999762e-01, 2.4038156e-06],
       [9.9929345e-01, 7.0652744e-04],
       [1.0000000e+00, 2.1878902e-08],
       ...,
       [9.9999982e-01, 1.6502962e-07],
       [9.9998951e-01, 1.0484842e-05],
       [9.8847032e-01, 1.1529699e-02]], dtype=float32)

In [23]:
y_val_pred = model.predict(X_val)
y_val_pred

array([0, 0, 0, ..., 0, 0, 0])

In [24]:
metrics.f1_score(y_val, y_val_pred)

0.6223175965665235

# Test scoring

In [25]:
y_test_prob = model.predict_proba(X_test)
y_test_prob

array([[1.3649464e-04, 9.9986351e-01],
       [9.9999470e-01, 5.3153567e-06],
       [9.9993867e-01, 6.1330204e-05],
       ...,
       [1.0000000e+00, 1.7484753e-08],
       [1.0000000e+00, 3.9570507e-09],
       [1.0000000e+00, 7.2811812e-10]], dtype=float32)

In [26]:
y_test_pred = model.predict(X_test)
y_test_pred

array([1, 0, 0, ..., 0, 0, 0])

In [27]:
test_pred = pd.Series(y_test_pred, index=X_test.index, name=target)
test_pred

id
39162    1
39163    0
39164    0
39165    1
39166    0
        ..
52341    0
52342    0
52343    0
52344    0
52345    0
Name: buy, Length: 13184, dtype: int64

In [28]:
test_pred.to_csv("Outputs/test-xgb-v2.csv")